In [227]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise import Reader
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import NMF
from surprise import CoClustering
from surprise import SlopeOne
from collections import defaultdict
import json

In [228]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [229]:
#HOW BIG DO WE WANT OUR UNIVERSE OF PRODUCTS TO BE
prodnum = 125

In [230]:
df = pd.read_parquet("C:/Users/jaycr/Downloads/sample_txn_data_2022.parquet")
df.drop_duplicates(inplace=True)
df.head(10)

,dw_gc_header,business_date,fiscal_week_end,fiscal_week,fiscal_year,daypart_name,lineitem_sequence,lineitem_seq_parent,lineitem_description,parent_product_description,...,actmodqty,actdiscqty,actprodprice,actgrosssales,actnetsales,actpromosales,actdiscsales,acttax,dw_gc_header_2,token_primary_account_identifier
0,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,999,999,TAX-LINE,None,...,0.0,0,0.00,0.00,0.00,0.00,0.00,0.28,42188263563,0880344659239910
1,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,1,1,NON-COMBO-ITEM,Beefy 5-Layer Burrito,...,0.0,0,2.99,2.99,2.99,0.00,0.00,0.00,42188263563,0880344659239910
2,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,2,2,NON-COMBO-ITEM,Cheesy Bean and Rice Burrito,...,0.0,0,1.29,1.29,1.29,0.00,0.00,0.00,42188263563,0880344659239910
3,42201144646,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,BREAKFAST,999,999,TAX-LINE,None,...,0.0,0,0.00,0.00,0.00,0.00,0.00,0.31,42201144646,8819419820104328
4,42201144646,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,BREAKFAST,1,1,NON-COMBO-ITEM,Breakfast Crunchwrap - Bacon,...,0.0,0,3.49,3.49,3.49,0.00,0.00,0.00,42201144646,8819419820104328
5,42197843459,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,LUNCH,999,999,TAX-LINE,None,...,0.0,0,0.00,0.00,0.00,0.00,0.00,0.11,42197843459,8456488655739252
6,42197843459,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,LUNCH,1,1,NON-COMBO-ITEM,Cheesy Fiesta Potatoes,...,0.0,0,1.89,1.89,1.89,0.00,0.00,0.00,42197843459,8456488655739252
7,42188307845,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,AFTERNOON,999,999,TAX-LINE,None,...,0.0,0,0.00,0.00,0.00,0.00,0.00,1.01,42188307845,8504411087099781
8,42188307845,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,AFTERNOON,1,1,NON-COMBO-ITEM,Chicken Quesadilla,...,0.0,0,4.99,14.97,14.97,0.00,0.00,0.00,42188307845,8504411087099781
9,42188307845,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,AFTERNOON,2,2,NON-COMBO-ITEM,Soft Taco,...,0.0,0,1.89,1.89,1.89,0.00,0.00,0.00,42188307845,8504411087099781


In [231]:
#grab product codes/descriptions for future labeling
parents_df = df[["parent_product_code", "parent_product_description"]]
parents_df = parents_df.rename(columns = {"parent_product_code":"product_code","parent_product_description":"product_description"})
parents_df["isModification"] = 0

children_df = df[["child_product_code", "child_product_description"]]
children_df = children_df.rename(columns = {"child_product_code":"product_code","child_product_description":"product_description"})
children_df["isModification"] = 0

mod_df = df[["modification_product_code", "modification_description"]]
mod_df = mod_df.rename(columns = {"modification_product_code":"product_code","modification_description":"product_description"})
mod_df["isModification"] = 1

prod_df = pd.concat([parents_df, children_df, mod_df])
prod_df = prod_df.drop_duplicates(subset='product_code')
prod_df = prod_df[prod_df["product_code"]!= "N/A"]


In [232]:
#data cleaning
#ditch any modifications
df = df[df["modification_product_code"]=="N/A"]
df["actgrosssales"] = df["actgrosssales"].astype("float")
df.head(5)

,dw_gc_header,business_date,fiscal_week_end,fiscal_week,fiscal_year,daypart_name,lineitem_sequence,lineitem_seq_parent,lineitem_description,parent_product_description,...,actmodqty,actdiscqty,actprodprice,actgrosssales,actnetsales,actpromosales,actdiscsales,acttax,dw_gc_header_2,token_primary_account_identifier
0,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,999,999,TAX-LINE,None,...,0.0,0,0.00,0.00,0.00,0.00,0.00,0.28,42188263563,0880344659239910
1,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,1,1,NON-COMBO-ITEM,Beefy 5-Layer Burrito,...,0.0,0,2.99,2.99,2.99,0.00,0.00,0.00,42188263563,0880344659239910
2,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,2,2,NON-COMBO-ITEM,Cheesy Bean and Rice Burrito,...,0.0,0,1.29,1.29,1.29,0.00,0.00,0.00,42188263563,0880344659239910
3,42201144646,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,BREAKFAST,999,999,TAX-LINE,None,...,0.0,0,0.00,0.00,0.00,0.00,0.00,0.31,42201144646,8819419820104328
4,42201144646,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,BREAKFAST,1,1,NON-COMBO-ITEM,Breakfast Crunchwrap - Bacon,...,0.0,0,3.49,3.49,3.49,0.00,0.00,0.00,42201144646,8819419820104328


In [233]:
#split as of 9/30
df_train = df[df["business_date"]<='2022-09-30']
df_val = df[df["business_date"]>'2022-09-30']

In [234]:
#find the universe of top products
prod_by_rev = df_train.groupby("child_product_code").agg({"actgrosssales":"sum"})
prod_by_rev.reset_index(inplace=True)
prod_by_rev.sort_values(by="actgrosssales",inplace=True, ascending=False)
prod_by_rev = prod_by_rev[prod_by_rev["child_product_code"]!= "TBFUND"]
prod_universe = prod_by_rev.head(prodnum)
prod_universe


,child_product_code,actgrosssales
478,Q-CK,126615.45
162,CMCQUE,108260.84
161,CMCHLP,79222.19
521,S-BF,75058.29
37,B-5LYR,66739.28
...,...,...
63,BCMHBB,2393.48
679,THB-BC,2351.95
45,B-CKSP,2280.80
483,QB-BN,2237.98


In [235]:
#build dataframe showing user, item, & % of time item is in order
#get list of users
products_by_user = df.drop_duplicates(subset="token_primary_account_identifier")["token_primary_account_identifier"].to_frame()
#cross join to generate 1 row per customer + product pairing
products_by_user = products_by_user.merge(prod_universe["child_product_code"],how='cross')

#Get number of orders per user
tot_orders_df = df.drop_duplicates(subset=["token_primary_account_identifier","dw_gc_header"]).groupby("token_primary_account_identifier").agg({"dw_gc_header":"count"})
tot_orders_df.reset_index(inplace=True)
tot_orders_df.rename(columns={"dw_gc_header":"# of Orders"},inplace=True)
products_by_user = products_by_user.merge(tot_orders_df, on = "token_primary_account_identifier", how="inner")

#Get how many times a user ordered each item
#cull original df down to just our selected universe of products
filter_df = df_train.merge(prod_universe["child_product_code"], on="child_product_code", how="inner")
orders_by_user = filter_df.groupby(["token_primary_account_identifier", "child_product_code"]).agg({"dw_gc_header":"nunique"})
orders_by_user.reset_index(inplace=True)
orders_by_user.rename(columns={"dw_gc_header":"# Order Inclusions"},inplace=True)
products_by_user = products_by_user.merge(orders_by_user, on = ["token_primary_account_identifier","child_product_code"], how = "inner")
products_by_user.fillna(value=0,inplace=True)
products_by_user["% Inclusion"] = products_by_user["# Order Inclusions"] / products_by_user["# of Orders"]
products_by_user["isActual"] = 1 
products_by_user.head(50)


,token_primary_account_identifier,child_product_code,# of Orders,# Order Inclusions,% Inclusion,isActual
0,0880344659239910,S-BF,10,1,0.100000,1
1,0880344659239910,B-5LYR,10,7,0.700000,1
2,0880344659239910,NBG-BF,10,6,0.600000,1
3,0880344659239910,GDCRUN,10,5,0.500000,1
4,0880344659239910,NACHOS,10,4,0.400000,1
5,0880344659239910,PIZ-BF,10,1,0.100000,1
6,0880344659239910,BCBRC,10,1,0.100000,1
7,0880344659239910,S-CTCH,10,1,0.100000,1
8,8819419820104328,B-5LYR,24,3,0.125000,1
9,8819419820104328,T-BF,24,2,0.083333,1


In [236]:
reader = Reader(rating_scale=(0,1))
data = Dataset.load_from_df(products_by_user[["token_primary_account_identifier","child_product_code","% Inclusion"]],reader)

In [237]:
#sim_options = {
#    "name": "cosine",
#    "user_based": False,  # Compute  similarities between items
#}
#algo = KNNWithMeans(sim_options=sim_options)
#trainSet = data.build_full_trainset()

In [251]:
#algo = SVD(verbose=True, n_epochs=10)
#sim_options = {
 #   "name": "pearson_baseline",
#    "user_based": False,  # Compute  similarities between items
#}
#algo = KNNBaseline(sim_options=sim_options)
#algo = NMF()
algo = CoClustering()
trainSet = data.build_full_trainset()

In [252]:
algo.fit(trainSet)

In [240]:
testset = trainSet.build_anti_testset()
predictions = algo.test(testset)

In [241]:
top_n = get_top_n(predictions, n=10)

In [242]:
x=0
for uid, user_ratings in top_n.items():
    x+=1
    if x <=10:
        print(uid, [iid for (iid, _) in user_ratings])

0880344659239910 ['CB-WRP', 'BEAN', 'QB-ST', 'C-BBSP', 'T-BF', 'BX-BYO', 'Q-CK', 'T-BFSP', 'LGBBZR', 'CHZROL']
8819419820104328 ['S-BF', 'BX-BYO', 'Q-CK', 'CB-WRP', 'QB-ST', 'SPTPPK', 'LGBLST', 'CHZROL', 'Q-ST', 'Q-CHZ']
8456488655739252 ['BEAN', 'C-BBSP', 'BX-BYO', 'B-5LYR', 'Q-CK', 'PNCHZ', 'CC-BUR', 'CB-WRP', 'LG-DIT', 'QB-ST']
8504411087099781 ['BEAN', 'T-BF', 'B-5LYR', 'BX-BYO', 'QB-ST', 'C-BBSP', 'CHZROL', 'CB-WRP', 'T-BFSP', 'GDCRUN']
0761848206682093 ['S-BF', 'BX-BYO', 'CB-WRP', 'Q-ST', 'CC-BUR', 'T-BFSP', 'QB-ST', 'TNC-LC', 'C-BBSP', 'P-CHFS']
0674945516353208 ['QB-ST', 'C-BBSP', 'B-5LYR', 'T-BF', 'LGBLST', 'Q-CK', 'PNCHZ', 'BXBLBK', 'CC-BUR', 'THB-BC']
0503885983932948 ['BEAN', 'BX-BYO', 'Q-CK', 'CB-WRP', 'CHZROL', 'CC-BUR', 'PNCHZ', 'P-CHFS', 'C-BBSP', 'Q-ST']
807246599191004 ['BEAN', 'T-BF', 'C-BBSP', 'QB-BF', 'S-BF', 'QB-BN', 'T-BFSP', 'BX-BYO', 'CHZROL', 'SPTPPK']
0915763119368468 ['CB-WRP', 'BX-BYO', 'Q-CK', 'QB-BN', 'CC-BUR', 'T-BFSP', 'LGBLST', 'QB-ST', 'Q-ST', 'BXBLBK

In [243]:
predictions_df = pd.DataFrame(columns=["token_primary_account_identifier", "child_product_code"	, "# of Orders"	, "# Order Inclusions",	"% Inclusion",	"isActual"])
prediction_list = []
for uid, user_ratings in top_n.items():
    for pred_item in user_ratings:
        prediction_list.append([uid, pred_item[0],np.nan,np.nan,pred_item[1],0])
predictions_df = pd.DataFrame(columns=["token_primary_account_identifier", "child_product_code"	, "# of Orders"	, "# Order Inclusions",	"% Inclusion",	"isActual"], data=prediction_list)
predictions_df.head(10)


,token_primary_account_identifier,child_product_code,# of Orders,# Order Inclusions,% Inclusion,isActual
0,0880344659239910,CB-WRP,NaN,NaN,0.333001,0
1,0880344659239910,BEAN,NaN,NaN,0.332575,0
2,0880344659239910,QB-ST,NaN,NaN,0.332330,0
3,0880344659239910,C-BBSP,NaN,NaN,0.332111,0
4,0880344659239910,T-BF,NaN,NaN,0.331890,0
5,0880344659239910,BX-BYO,NaN,NaN,0.328640,0
6,0880344659239910,Q-CK,NaN,NaN,0.328191,0
7,0880344659239910,T-BFSP,NaN,NaN,0.327625,0
8,0880344659239910,LGBBZR,NaN,NaN,0.327491,0
9,0880344659239910,CHZROL,NaN,NaN,0.326587,0


In [244]:
final_rec_df = pd.concat([products_by_user, predictions_df])
final_rec_df.sort_values(by=["token_primary_account_identifier","% Inclusion"],ascending=[True,False],inplace=True)
final_rec_df.head(10)

,token_primary_account_identifier,child_product_code,# of Orders,# Order Inclusions,% Inclusion,isActual
160657,0000129864897916,LGBLST,11.0,10.0,0.909091,1
160655,0000129864897916,CR-WRP,11.0,9.0,0.818182,1
101560,0000129864897916,T-BF,NaN,NaN,0.256279,0
101561,0000129864897916,CB-WRP,NaN,NaN,0.252817,0
101562,0000129864897916,QB-ST,NaN,NaN,0.251457,0
101563,0000129864897916,Q-CK,NaN,NaN,0.251382,0
101564,0000129864897916,CC-BUR,NaN,NaN,0.250948,0
101565,0000129864897916,BX-BYO,NaN,NaN,0.250772,0
101566,0000129864897916,CMNBGR,NaN,NaN,0.250152,0
101567,0000129864897916,C-BBSP,NaN,NaN,0.248956,0


In [245]:
#Create Dataframe showing user, actual items ordered, and our top predictions/recommendations
#Pull Items Ordered
df_val = pd.merge(df_val,prod_universe["child_product_code"],how="inner",on="child_product_code").drop_duplicates(subset=["dw_gc_header","child_product_code"])
df_val.head(10)

next_transaction = df_val[df_val["business_date"]>'2022-09-30'].drop_duplicates(subset="dw_gc_header").sort_values("business_date",ascending=True).groupby("token_primary_account_identifier").head(1)[["token_primary_account_identifier","dw_gc_header"]]
next_transaction.head(10)

next_trans_items =df_val.groupby('dw_gc_header')["child_product_code"].apply(lambda x: "%s" % ', '.join(x)).to_frame()
next_trans_items.reset_index(inplace=True)
next_trans_items.rename(columns={"child_product_code":"items_ordered"},inplace=True)
next_transaction = next_transaction.merge(next_trans_items, how='inner',on='dw_gc_header')

#Pull top 10 predictions
top10pred = final_rec_df.groupby('token_primary_account_identifier')[["token_primary_account_identifier","child_product_code"]].head(10)
top10pred = top10pred.groupby('token_primary_account_identifier')["child_product_code"].apply(lambda x: "%s" % ', '.join(x)).to_frame()
top10pred.reset_index(inplace=True)
top10pred.rename(columns={"child_product_code":"Recommendation Basket"},inplace=True)
top10pred.head(10)

next_transaction = next_transaction.merge(top10pred, how='inner', on='token_primary_account_identifier')
next_transaction.head(10)

,token_primary_account_identifier,dw_gc_header,items_ordered,Recommendation Basket
0,0800738988499795,48231856946,"C-BFSP, B-5LYR","C-BBSP, BEAN, T-BF, S-BF, T-BFSP, TGC-NC, CB-W..."
1,2175923333040678,48231811531,"C-BFSP, T-BFSP","C-BFSP, T-BFSP, CMCHLP, TNC-SP, LG-DRP, BEAN, ..."
2,522226,48237576084,"B-GRCZ, T-BF, CNBN12","NACHOS, C-BBSP, CB-WRP, BX-BYO, BEAN, CC-BUR, ..."
3,0829983154363342,48258538854,"B-GRCZ, T-BF, LG-DEW, CR-WRP, CMBTBF","NACHOS, T-BF, NBG-BF, C-BFSP, CINN-2, B-5LYR, ..."
4,8185961687590855,48231829383,"B-GRCZ, P-CHFS, LG-DEW, CINN-2, GDCRUN, CHZROL","CHZROL, LG-DEW, GDCRUN, CINN-2, C-BFSP, ML-CKC..."
5,8874027209980117,48226173795,B-GRCZ,"B-5LYR, B-GRCZ, PIZ-BF, B-CDBC, T-BF, BEAN, C-..."
6,0980325321505972,48219885764,"B-GRCZ, P-CHFS, S-BF, CHZROL, LGMIST","CHZROL, S-BF, BCBRC, P-CHFS, B-GRCZ, C-BBSP, B..."
7,0693832184888696,48258355358,"B-GRCZ, PIZ-BF, BWLPST","LG-PEP, B-BFSP, Q-ST, T-BFSP, B-CDBS, S-BF, CM..."
8,2028601612094613,48255424561,"B-GRCZ, PIZ-BF","BEAN, T-BF, CHZROL, LGBBZR, Q-CK, CMCQUE, QB-S..."
9,8906214557915420,48231635507,"B-GRCZ, PIZ-BF, TWIST, GDCRUN","T-BF, B-BFML, Q-CK, PIZ-BF, BEAN, QB-ST, C-BBS..."


In [246]:
#create dictionaries of recommendations and orders
ordered_dict = next_transaction[['token_primary_account_identifier','items_ordered']].set_index('token_primary_account_identifier').to_dict('index')
user_rec_dict = next_transaction[['token_primary_account_identifier','Recommendation Basket']].set_index('token_primary_account_identifier').to_dict('index')

In [247]:
in_order_dict = {}
for k, v in user_rec_dict.items():
    in_order_dict[k] = np.zeros(10)
    for i in range (0,10):
        if v['Recommendation Basket'].split(", ")[i] in ordered_dict[k]['items_ordered'].split(", "):
            in_order_dict[k][i] = 1
#test case
print(user_rec_dict["0012691225416239"]['Recommendation Basket'])
print(ordered_dict["0012691225416239"]['items_ordered'])
print(in_order_dict["0012691225416239"])

CR-WRP, TNC-LC, BEAN, T-BF, Q-CK, T-BFSP, S-BF, C-CKSP, BXCHRB, B-5LYR
PIZ-BF, CR-WRP, TWIST
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [248]:
next_transaction["Rec in Order"] = next_transaction["token_primary_account_identifier"].map(in_order_dict)
next_transaction.head(10)

,token_primary_account_identifier,dw_gc_header,items_ordered,Recommendation Basket,Rec in Order
0,0800738988499795,48231856946,"C-BFSP, B-5LYR","C-BBSP, BEAN, T-BF, S-BF, T-BFSP, TGC-NC, CB-W...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2175923333040678,48231811531,"C-BFSP, T-BFSP","C-BFSP, T-BFSP, CMCHLP, TNC-SP, LG-DRP, BEAN, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,522226,48237576084,"B-GRCZ, T-BF, CNBN12","NACHOS, C-BBSP, CB-WRP, BX-BYO, BEAN, CC-BUR, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0829983154363342,48258538854,"B-GRCZ, T-BF, LG-DEW, CR-WRP, CMBTBF","NACHOS, T-BF, NBG-BF, C-BFSP, CINN-2, B-5LYR, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,8185961687590855,48231829383,"B-GRCZ, P-CHFS, LG-DEW, CINN-2, GDCRUN, CHZROL","CHZROL, LG-DEW, GDCRUN, CINN-2, C-BFSP, ML-CKC...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,8874027209980117,48226173795,B-GRCZ,"B-5LYR, B-GRCZ, PIZ-BF, B-CDBC, T-BF, BEAN, C-...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,0980325321505972,48219885764,"B-GRCZ, P-CHFS, S-BF, CHZROL, LGMIST","CHZROL, S-BF, BCBRC, P-CHFS, B-GRCZ, C-BBSP, B...","[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
7,0693832184888696,48258355358,"B-GRCZ, PIZ-BF, BWLPST","LG-PEP, B-BFSP, Q-ST, T-BFSP, B-CDBS, S-BF, CM...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
8,2028601612094613,48255424561,"B-GRCZ, PIZ-BF","BEAN, T-BF, CHZROL, LGBBZR, Q-CK, CMCQUE, QB-S...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,8906214557915420,48231635507,"B-GRCZ, PIZ-BF, TWIST, GDCRUN","T-BF, B-BFML, Q-CK, PIZ-BF, BEAN, QB-ST, C-BBS...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [249]:
for n in range(1,11):
    next_transaction["Basket Size " + str(n) + " Success"] = next_transaction['Rec in Order'].apply(lambda x:x[0:n].max())

In [250]:
final_tally = next_transaction.agg({"token_primary_account_identifier":"count","Basket Size 1 Success":"sum","Basket Size 2 Success":"sum","Basket Size 3 Success":"sum","Basket Size 4 Success":"sum","Basket Size 5 Success":"sum","Basket Size 6 Success":"sum",
                                "Basket Size 7 Success":"sum","Basket Size 8 Success":"sum","Basket Size 9 Success":"sum","Basket Size 10 Success":"sum"})
final_tally = final_tally.to_frame().reset_index()#.transpose()
final_tally.rename(columns={0:"value"},inplace=True)
tot = final_tally[final_tally["index"]=="token_primary_account_identifier"]["value"][0]
final_tally["% Accuracy"] = final_tally["value"] / tot
final_tally

,index,value,% Accuracy
0,token_primary_account_identifier,13456.0,1.000000
1,Basket Size 1 Success,6915.0,0.513897
2,Basket Size 2 Success,8455.0,0.628344
3,Basket Size 3 Success,9089.0,0.675461
4,Basket Size 4 Success,9565.0,0.710835
5,Basket Size 5 Success,9842.0,0.731421
6,Basket Size 6 Success,10058.0,0.747473
7,Basket Size 7 Success,10235.0,0.760627
8,Basket Size 8 Success,10380.0,0.771403
9,Basket Size 9 Success,10502.0,0.780470
